In [1]:
# Basic RNNs in Tensorflow
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
n_inputs = 3
n_neurons = 5

x0 = tf.placeholder(tf.float32, [None, n_inputs])
x1 = tf.placeholder(tf.float32, [None, n_inputs])

wx = tf.Variable(tf.random_normal(shape = [n_inputs, n_neurons], dtype = tf.float32))
wy = tf.Variable(tf.random_normal(shape = [n_neurons, n_neurons], dtype = tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype = tf.float32))

y0 = tf.tanh(tf.matmul(x0, wx) +b)
y1 = tf.tanh(tf.matmul(y0, wy)+ tf.matmul(x1, wx) +b)

init = tf.global_variables_initializer()

In [3]:
import numpy as np

#Mini-batch 
x0_batch = np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]]) #t = 0
x1_batch = np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]]) #t = 1

with tf.Session() as sess:
    init.run()
    y0_val, y1_val = sess.run([y0, y1], feed_dict = {x0:x0_batch, x1:x1_batch})
    
    

In [4]:
print(y0_val) #output at t = 0

[[-0.9293567  -0.8781928  -0.96271807 -0.39214566 -0.8828403 ]
 [-0.9978528   0.8130378  -0.99999803  0.81579167 -0.9916774 ]
 [-0.9999369   0.99862164 -1.          0.9910529  -0.99943894]
 [-0.9551768   1.         -0.6346127   1.          0.9995964 ]]


In [5]:
print(y1_val) #output at t = 1

[[-0.9999985   0.9999997  -1.          0.99999964 -0.9293559 ]
 [-0.968734   -0.99417424  0.4295082   0.73013866 -0.6252687 ]
 [-0.9989001   0.9969851  -0.99999934  0.99957544 -0.97793955]
 [ 0.94890285 -0.4627683   0.41344348  0.9921769  -0.4482873 ]]


In [6]:
# Building the same model using Tensorflow's RNN operations.
x0 = tf.placeholder(tf.float32, [None, n_inputs])
x1 = tf.placeholder(tf.float32, [None, n_inputs])


basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units = n_neurons)
output_seqs, state = tf.nn.static_rnn(basic_cell, [x0, x1], dtype = tf.float32)


Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
y0, y1 = output_seqs

In [9]:
# re-write (better)
n_steps = 2
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
X_seqs = tf.unstack(tf.transpose(X, perm = [1,0,2]))
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units = n_neurons, reuse = True)
output_seqs, state = tf.nn.static_rnn(basic_cell, X_seqs, dtype = tf.float32)
outputs = tf.transpose(tf.stack(output_seqs), perm = [1,0,2])

In [16]:
x_batch = np.array([
    #t = 0      t =1
    [[0,1,2],[9,8,7]],
    [[3,4,5],[0,0,0]],
    [[6,7,8],[6,5,4]],
    [[9,8,1],[3,2,1]],
])
x_batch.shape

(4, 2, 3)

In [17]:
x0_batch.shape

(4, 3)

In [24]:

with tf.Session() as sess:
    pass
    #init.run()
    #output_val = outputs.eval(feed_dict = {X:x_batch})

   

This approach still builds a graph containing one cell per time step. If there were 50 time steps, the graph would look pretty ugly. With such a large graph, you may even get out-of-memory (OOM) errors during backpropagation (especially with the limited memory of GPU cards), since it must store all tensor values during the forward pass so it can use them to compute gradient during the reverse pass.

# dynamic unrolling through time

The dynamic_rnn() function uses a while loop operation to run over the ell the appropriate number of times, and you can set swap_memory = True if you want it to swap the GPU's memory to the CPU's memory during backpropagation to avoid OOM errors.

Conviniently, it also accepts a single tensor for all inputs at every time step (shape [None, n_steps, n_inputs]) and it outputs a single tensor for all outputs at every time step (shape [none, n_steps, n_neurons]), there is no need to stack, unstack or transpose.

In [26]:
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units = n_neurons, reuse = True)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype = tf.float32)


Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


so far we have used only-fixed size input sequences (all exactly two steps long). what if the input sequences have variable lengths, (e.g., like sentences)? In this case you should set the sequence_length parameter when calling the dynamic_rnn() (or static_rnn()) function; it must be a 1D tensor indicating the length of the input sequence for each instance. For example, 


In [27]:
seq_length = tf.placeholder(tf.int32, [None])
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units = n_neurons, reuse = True)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X,
                                    dtype = tf.float32,
                                   sequence_length = seq_length)


for example, suppose the second input sequence contains only one input instead of two. It must be padded with zero vector in order to fit in the input tensor X (because the input tensor's second dimension is the size of the longest sequence--i.e., 2).


In [28]:
seq_length_batch = np.array([2,1,2,2])

In [30]:
with tf.Session() as sess:
    init.run()
    #outputs_val, states_val = sess.run([outputs, states],
    #                                  feed_dict = {X:x_batch, seq_length:seq_length_batch})

To handle variable-length output sequences

In this case, most common solution is to define a special output called an end-of-sequence token (EOS token). Any output past the EOS should be ignored.